In [3]:
import sys
sys.path.append('/home/lev/object-centric/edge-generation/models/')
import torch_geometric
import torch
from omegaconf import OmegaConf
from dataloader import PL_DataModule
from models.model import Network

import hydra

from hydra import compose, initialize
from omegaconf import OmegaConf
from sklearn.metrics import confusion_matrix
import numpy as np

initialize(config_path="configs/", job_name="test_app")


hydra.initialize()

In [6]:
cfg = compose(config_name="defaults")
pl = PL_DataModule(cfg.dataloader)
batch = next(iter(pl.train_dataloader()))
# Obtain feature sizes and number of labels

cfg.model.insize = batch.x.shape[1]
cfg.model.outsize = torch.unique(batch.y).shape[0]


model_classic = Network(cfg.model)
res_cl = model_classic.loss_function(batch)


cfg = compose(config_name="defaults")
cfg.model.insize = batch.x.shape[1]
cfg.model.outsize = torch.unique(batch.y).shape[0]
cfg.model.edge_generation_type = 'DynamicEdgeConv_DGM'
model_dcg = Network(cfg.model)
res_dcg = model_dcg.loss_function(batch)

res_cl['loss'].backward()
res_dcg['loss'].backward()
# res['loss'].backward()
# for name,i in model.model.named_parameters():
#     if 'temperature' in name:
#         print(i)
    

/opt/anaconda3/envs/lev_eg/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/opt/anaconda3/envs/lev_eg/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


AttributeError: 'Network' object has no attribute 'device'

In [ ]:
def weight_vector(y, preds):
    cm = confusion_matrix(y, preds)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    class_acc = torch.tensor(cm.diagonal())    

    y = y.view(1, -1)
    preds = preds.view(1, -1)
    class_acc = class_acc[y]

    weight_matrix = ((y == preds) * (class_acc-1) + (y != preds)*class_acc)
    return weight_matrix

y = torch.tensor(np.random.binomial(10, 0.3, 455)) 
preds = torch.tensor(np.random.binomial(10, 0.3, 455)) 

prob_L = torch.cat([model_dcg.model.model[i].edge_conv.probs.unsqueeze(0) for i in range(len(model_dcg.model.model))], dim=0)
mask_L = torch.cat([(model_dcg.model.model[i].edge_conv.mask * 1).unsqueeze(0) for i in range(len(model_dcg.model.model))], dim=0)

w_vec = weight_vector(y, preds)

weight_mask = (prob_L * mask_L) * w_vec 
negative_mask = (mask_L == 0 ) * 1
weight_mask = weight_mask + negative_mask
weight_mask = torch.prod(weight_mask, dim=1)
weight_mask.prod(dim=0).sum()



/tmp/ipykernel_1745211/2661014610.py:3: RuntimeWarning: invalid value encountered in true_divide
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


tensor(2.1133, dtype=torch.float64, grad_fn=<SumBackward0>)

In [113]:
a = {3:2, 2:2}
b = {5:2, 2:2}
dict(list(a.items()) + list(b.items()))

{3: 2, 2: 2, 5: 2}

In [5]:
y.device()

NameError: name 'y' is not defined

In [9]:
batch.x.device

device(type='cpu')

In [6]:
print(weight_vector(a, b)[0,:10])
print(a[:10])
print(b[:10])

tensor([0.1333, 0.1800, 0.2783, 0.3440, 0.2250, 0.0714, 0.0400, 0.0000,    nan],
       dtype=torch.float64)
tensor([ 0.2250,  0.2250,  0.3440, -0.8200,  0.2250,  0.3440, -0.6560,  0.2250,
         0.1800,  0.3440], dtype=torch.float64)
tensor([4, 4, 3, 1, 4, 3, 3, 4, 1, 3])
tensor([3, 3, 5, 1, 2, 0, 3, 3, 4, 4])


/tmp/ipykernel_1745211/1371237923.py:6: RuntimeWarning: invalid value encountered in true_divide
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


In [40]:
torch

tensor([0.1333, 0.1800, 0.2783, 0.3440, 0.2250, 0.0714, 0.0400, 0.0000,    nan],
       dtype=torch.float64)


/tmp/ipykernel_1745211/1371237923.py:6: RuntimeWarning: invalid value encountered in true_divide
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


tensor(-0.1573, dtype=torch.float64, grad_fn=<SumBackward0>)

In [88]:
torch.sum(torch.Tensor(10,5), dim=0).shape

torch.Size([5])

In [85]:
torch.Tensor(10,5)

tensor([[4.4773e-04, 3.0799e-41, 5.7232e-13, 3.0799e-41, 1.1210e-43],
        [0.0000e+00, 8.9683e-44, 0.0000e+00, 5.0083e-04, 3.0799e-41],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.4013e-45, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 9.1084e-44, 0.0000e+00, 4.4773e-04],
        [3.0799e-41, 5.0081e-04, 3.0799e-41, 5.8855e-44, 0.0000e+00],
        [4.6243e-44, 0.0000e+00, 4.4790e-04, 3.0799e-41, 5.7231e-13],
        [3.0799e-41, 2.9147e-43, 0.0000e+00, 1.1210e-43, 0.0000e+00],
        [4.3079e-12, 3.0799e-41, 0.0000e+00, 0.0000e+00, 1.4013e-45],
        [0.0000e+00, 0.0000e+00, 4.7851e+22, 6.1948e-04, 2.5749e-12]])

In [73]:
probs

tensor([[[1.0000e+00, 1.0000e-06, 7.4016e-02,  ..., 1.0000e-06,
          1.0000e-06, 1.0000e-06],
         [1.0000e-06, 1.0000e+00, 1.0000e-06,  ..., 1.0000e-06,
          1.0000e-06, 1.0000e-06],
         [7.4016e-02, 1.0000e-06, 1.0000e+00,  ..., 1.0000e-06,
          1.0000e-06, 1.0000e-06],
         ...,
         [1.0000e-06, 1.0000e-06, 1.0000e-06,  ..., 1.0000e+00,
          1.0000e-06, 1.0000e-06],
         [1.0000e-06, 1.0000e-06, 1.0000e-06,  ..., 1.0000e-06,
          1.0000e+00, 1.0000e-06],
         [1.0000e-06, 1.0000e-06, 1.0000e-06,  ..., 1.0000e-06,
          1.0000e-06, 1.0000e+00]],

        [[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00],
         [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00],
         [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00],
         ...,
         [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.000

In [71]:
torch.where(maskes>0)[0]

tensor([0, 0, 0,  ..., 1, 1, 1])

In [72]:
maskes

tensor([[[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 0]]])

In [223]:
def generate_matrix2(y, preds):
    cm = confusion_matrix(y, preds)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    class_acc = cm.diagonal()

    matrix_y = y.view(1, 1, -1).repeat((1, batch.x.shape[0], 1))
    matrix_pred = preds.view(1, 1, -1).repeat((1, batch.x.shape[0], 1))

    unique_labels = torch.unique(y).view(-1, 1, 1)
    class_acc = torch.tensor(class_acc).view(-1, 1, 1)
    
    class_acc = class_acc[matrix_y].view(1, batch.x.shape[0], batch.x.shape[0])
    #matrix_y = (matrix_y == unique_labels) * class_acc
    
    weight_matrix = ((matrix_y == matrix_pred) * (class_acc-1) + (matrix_y != matrix_pred)*class_acc)

    # matrix_pred = (matrix_pred == unique_labels) * (-1)
    # weight_matrix = (matrix_pred + matrix_y).sum(dim=0)
    return weight_matrix, matrix_y, class_acc

In [243]:
def generate_matrix2(y, preds):
    cm = confusion_matrix(y, preds)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    class_acc = cm.diagonal()

    matrix_y = y.view(1,-1)
    matrix_pred = preds.view(1,-1)

    unique_labels = torch.unique(y).view(-1, 1)
    class_acc = torch.tensor(class_acc).view(-1, 1)
    print(class_acc)
    #class_acc = class_acc[matrix_y].view(1,455,455)
    #matrix_y = (matrix_y == unique_labels) * class_acc
    class_acc = class_acc[matrix_y]
    weight_matrix = ((matrix_y == matrix_pred) * (class_acc-1) + (matrix_y != matrix_pred)*class_acc)

    # matrix_pred = (matrix_pred == unique_labels) * (-1)
    # weight_matrix = (matrix_pred + matrix_y).sum(dim=0)
    return weight_matrix, matrix_y, class_acc

In [266]:
weight_matrix[:, :10]

tensor([[-0.8333,  0.1333, -0.7043,  0.1667,  0.2957,  0.0909,  0.0909,  0.2957,
         -0.7043,  0.1667]], dtype=torch.float64)

In [269]:
print(a[:10])
print(b[:10])

tensor([4, 6, 2, 4, 2, 1, 1, 2, 2, 4])
tensor([4, 3, 2, 6, 5, 4, 2, 4, 2, 2])


In [270]:
cm.diagonal()

array([0.        , 0.09090909, 0.29565217, 0.2519685 , 0.16666667,
       0.09756098, 0.13333333, 0.        , 0.        ])

In [271]:
1 -  0.16666667

0.83333333

In [244]:
weight_matrix, matrix_y, class_acc = generate_matrix2(a, b)

tensor([[0.0000],
        [0.0909],
        [0.2957],
        [0.2520],
        [0.1667],
        [0.0976],
        [0.1333],
        [0.0000],
        [0.0000]], dtype=torch.float64)


In [245]:
weight_matrix

tensor([[[-0.8333,  0.1667, -0.8333,  ..., -0.8333, -0.8333,  0.1667],
         [-0.8667,  0.1333, -0.8667,  ..., -0.8667, -0.8667,  0.1333],
         [-0.7043,  0.2957, -0.7043,  ..., -0.7043, -0.7043,  0.2957],
         ...,
         [-0.7043,  0.2957, -0.7043,  ..., -0.7043, -0.7043,  0.2957],
         [-0.7043,  0.2957, -0.7043,  ..., -0.7043, -0.7043,  0.2957],
         [-0.7043,  0.2957, -0.7043,  ..., -0.7043, -0.7043,  0.2957]]],
       dtype=torch.float64)

In [225]:
weight_matrix[:,:,:10]

tensor([[[-0.8333,  0.1333, -0.7043,  ...,  0.2957, -0.7043,  0.1667],
         [-0.8333,  0.1333, -0.7043,  ...,  0.2957, -0.7043,  0.1667],
         [-0.8333,  0.1333, -0.7043,  ...,  0.2957, -0.7043,  0.1667],
         ...,
         [-0.8333,  0.1333, -0.7043,  ...,  0.2957, -0.7043,  0.1667],
         [-0.8333,  0.1333, -0.7043,  ...,  0.2957, -0.7043,  0.1667],
         [-0.8333,  0.1333, -0.7043,  ...,  0.2957, -0.7043,  0.1667]]],
       dtype=torch.float64)

In [233]:
print(a[:10])
print(b[:10])

tensor([4, 6, 2, 4, 2, 1, 1, 2, 2, 4])
tensor([4, 3, 2, 6, 5, 4, 2, 4, 2, 2])


In [222]:
class_acc

tensor([[[0.1667, 0.1333, 0.2957,  ..., 0.2957, 0.2957, 0.2957],
         [0.1667, 0.1333, 0.2957,  ..., 0.2957, 0.2957, 0.2957],
         [0.1667, 0.1333, 0.2957,  ..., 0.2957, 0.2957, 0.2957],
         ...,
         [0.1667, 0.1333, 0.2957,  ..., 0.2957, 0.2957, 0.2957],
         [0.1667, 0.1333, 0.2957,  ..., 0.2957, 0.2957, 0.2957],
         [0.1667, 0.1333, 0.2957,  ..., 0.2957, 0.2957, 0.2957]]],
       dtype=torch.float64)

In [203]:
wm.shape

torch.Size([9, 455, 455])

In [187]:
generate_matrix2(a, b)[3]

tensor([[-0.7480,  0.2520, -0.7480,  ..., -0.7480, -0.7480,  0.2520],
        [-0.7480,  0.2520, -0.7480,  ..., -0.7480, -0.7480,  0.2520],
        [-0.7480,  0.2520, -0.7480,  ..., -0.7480, -0.7480,  0.2520],
        ...,
        [-0.7480,  0.2520, -0.7480,  ..., -0.7480, -0.7480,  0.2520],
        [-0.7480,  0.2520, -0.7480,  ..., -0.7480, -0.7480,  0.2520],
        [-0.7480,  0.2520, -0.7480,  ..., -0.7480, -0.7480,  0.2520]],
       dtype=torch.float64)

In [84]:
def generate_mask(y, preds):
    
    cm = confusion_matrix(y, preds)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    class_acc = cm.diagonal()
    print(class_acc)

    class_mask = y.unsqueeze(0).repeat((batch.x.shape[0],1))
    pred_mask = preds.unsqueeze(0).repeat((batch.x.shape[0],1)) 
    final_mask = torch.zeros((batch.x.shape[0], batch.x.shape[0]))
    for cl in range(len(class_acc)):
        class_mask_temp = (class_mask == cl) * class_acc[cl]
        pred_mask_temp = (pred_mask == cl) * (-1)
        final_mask += class_mask_temp + pred_mask_temp
    return final_mask

    

In [179]:
generate_matrix2(a, b)

tensor([[-7.9639,  1.0361, -7.9639,  ..., -7.9639, -7.9639,  1.0361],
        [-7.9639,  1.0361, -7.9639,  ..., -7.9639, -7.9639,  1.0361],
        [-7.9639,  1.0361, -7.9639,  ..., -7.9639, -7.9639,  1.0361],
        ...,
        [-7.9639,  1.0361, -7.9639,  ..., -7.9639, -7.9639,  1.0361],
        [-7.9639,  1.0361, -7.9639,  ..., -7.9639, -7.9639,  1.0361],
        [-7.9639,  1.0361, -7.9639,  ..., -7.9639, -7.9639,  1.0361]],
       dtype=torch.float64)

In [178]:
a = torch.tensor(np.random.binomial(10, 0.3, 455)) #torch.bernoulli(torch.ones(455)-0.5)
b = torch.tensor(np.random.binomial(10, 0.3, 455)) #torch.bernoulli(torch.ones(455)-0.5)
generate_mask(a, b)

[0.         0.09090909 0.29565217 0.2519685  0.16666667 0.09756098
 0.13333333 0.         0.        ]


tensor([[-0.8333, -0.8667, -0.7043,  ..., -0.7043, -0.7043, -0.7043],
        [-0.8333, -0.8667, -0.7043,  ..., -0.7043, -0.7043, -0.7043],
        [-0.8333, -0.8667, -0.7043,  ..., -0.7043, -0.7043, -0.7043],
        ...,
        [-0.8333, -0.8667, -0.7043,  ..., -0.7043, -0.7043, -0.7043],
        [-0.8333, -0.8667, -0.7043,  ..., -0.7043, -0.7043, -0.7043],
        [-0.8333, -0.8667, -0.7043,  ..., -0.7043, -0.7043, -0.7043]])

In [173]:
print(a[:10])
print(b[:10])

tensor([2, 3, 3, 2, 3, 3, 2, 3, 3, 5])
tensor([5, 3, 3, 5, 3, 2, 2, 6, 4, 3])


array([2, 5, 3, 3, 4, 3, 5, 1, 1, 1, 7, 7, 2, 3, 2, 8, 3, 3, 3, 4, 2, 3,
       5, 2, 3, 0, 2, 4, 2, 2, 2, 2, 4, 2, 4, 2, 4, 4, 2, 3, 1, 6, 4, 2,
       2, 1, 4, 0, 2, 0, 4, 3, 3, 5, 7, 2, 3, 3, 1, 5, 4, 6, 4, 1, 3, 2,
       1, 1, 4, 3, 1, 3, 4, 2, 1, 4, 6, 2, 5, 3, 2, 6, 6, 2, 5, 3, 4, 3,
       4, 2, 5, 2, 7, 2, 6, 4, 4, 2, 2, 5, 3, 5, 2, 3, 4, 1, 3, 5, 4, 1,
       5, 2, 3, 1, 2, 3, 3, 3, 5, 3, 3, 3, 3, 4, 4, 5, 2, 5, 4, 5, 3, 5,
       3, 3, 0, 3, 1, 2, 2, 1, 5, 2, 5, 2, 2, 4, 2, 2, 1, 5, 3, 4, 3, 1,
       2, 8, 3, 2, 3, 2, 3, 1, 4, 4, 2, 3, 3, 7, 3, 7, 2, 3, 2, 2, 3, 4,
       2, 3, 2, 3, 4, 3, 4, 3, 2, 2, 1, 3, 5, 2, 1, 1, 2, 3, 4, 4, 3, 2,
       2, 4, 2, 1, 4, 0, 3, 2, 1, 2, 2, 4, 3, 9, 2, 4, 3, 1, 2, 3, 4, 2,
       4, 2, 2, 2, 3, 3, 4, 4, 2, 3, 5, 3, 3, 4, 2, 6, 4, 6, 2, 4, 4, 1,
       4, 4, 2, 1, 1, 4, 3, 4, 4, 3, 3, 3, 3, 7, 1, 5, 1, 1, 4, 3, 4, 4,
       4, 4, 1, 2, 2, 5, 3, 9, 1, 1, 4, 1, 4, 3, 3, 1, 2, 1, 4, 3, 3, 4,
       2, 3, 2, 1, 2, 4, 2, 3, 3, 2, 4, 5, 1, 1, 4,

In [5]:
probs = model_dcg.model.model[-1].edge_conv.probs
mask = model_dcg.model.model[-1].edge_conv.mask*1


In [21]:
mask[model_dcg.model.model[-1].edge_conv.edge_index[0],model_dcg.model.model[-1].edge_conv.edge_index[1]].shape

torch.Size([910])

In [28]:
mask.sum(dim=0)


tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [36]:
mask

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [7]:
from sklearn.metrics import confusion_matrix

import numpy as np

y_true = [0, 1, 2, 2, 2]

y_pred = [0, 0, 2, 2, 1]





cm = confusion_matrix(y_true, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm.diagonal()

#array([1.        , 0.        , 0.66666667])

array([1.        , 0.        , 0.66666667])

In [ ]:
# for name,i in model_classic.model.named_parameters():
#     if i.grad is not None:
#         shape = i.grad.shape
#     else:
#         shape='None'
    
#     print(f"name={name}, shape = {shape}")

    
    

name=model.0.edge_conv.mlp.0.weight, shape = torch.Size([2, 60])
name=model.0.edge_conv.mlp.0.bias, shape = torch.Size([2])
name=model.0.edge_conv.mlp.2.weight, shape = torch.Size([2, 2])
name=model.0.edge_conv.mlp.2.bias, shape = torch.Size([2])
name=model.1.edge_conv.mlp.0.weight, shape = torch.Size([2, 4])
name=model.1.edge_conv.mlp.0.bias, shape = torch.Size([2])
name=model.1.edge_conv.mlp.2.weight, shape = torch.Size([2, 2])
name=model.1.edge_conv.mlp.2.bias, shape = torch.Size([2])


In [5]:
for name,i in model_dcg.model.named_parameters():
    if i.grad is not None:
        shape = i.grad.shape
    else:
        shape='None'
    
    print(f"name={name}, shape = {shape}")
    if "temperature" in name:
        print(f"name={name}, shape = {i.grad}")

name=model.0.edge_conv.temperature, shape = torch.Size([1])
name=model.0.edge_conv.temperature, shape = tensor([18.3348])
name=model.0.edge_conv.mlp.0.weight, shape = torch.Size([2, 60])
name=model.0.edge_conv.mlp.0.bias, shape = torch.Size([2])
name=model.0.edge_conv.mlp.2.weight, shape = torch.Size([2, 2])
name=model.0.edge_conv.mlp.2.bias, shape = torch.Size([2])
name=model.0.edge_conv.mlp_t.0.weight, shape = torch.Size([30, 30])
name=model.0.edge_conv.mlp_t.0.bias, shape = torch.Size([30])
name=model.1.edge_conv.temperature, shape = torch.Size([1])
name=model.1.edge_conv.temperature, shape = tensor([-2186.8000])
name=model.1.edge_conv.mlp.0.weight, shape = torch.Size([2, 4])
name=model.1.edge_conv.mlp.0.bias, shape = torch.Size([2])
name=model.1.edge_conv.mlp.2.weight, shape = torch.Size([2, 2])
name=model.1.edge_conv.mlp.2.bias, shape = torch.Size([2])
name=model.1.edge_conv.mlp_t.0.weight, shape = torch.Size([2, 2])
name=model.1.edge_conv.mlp_t.0.bias, shape = torch.Size([2])


In [55]:
import torch
import torch_cluster
from torch_geometric.nn import MessagePassing
from torch.nn import Sequential as Seq, Linear, ReLU

class EdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='mean') #  "Max" aggregation.
        self.mlp = Seq(Linear(2 * in_channels, out_channels),
                       ReLU(),
                       Linear(out_channels, out_channels))

        # self.mlp_k0 = Seq(Linear(in_channels, out_channels),
        #                ReLU(),
        #                Linear(out_channels, out_channels))

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        # x_i has shape [E, in_channels]
        # x_j has shape [E, in_channels] 
        # Workout the case when k=0, hence one node
        # if torch.all(x_j == x_i):
        #     return self.mlp_k0(x_i)

        tmp = torch.cat([x_i, x_j - x_i], dim=1)  # tmp has shape [E, 2 * in_channels]
        return self.mlp(tmp)


ec = EdgeConv(10,1)

a = torch.ones(10,10)*13
edge_index = torch.Tensor([[0,1,2,3,4,5,6,7,8,9], [0,1,2,3,4,5,6,7,8,9]]).to(torch.long)
res = ec(a, edge_index)
torch.sum(torch.ones(res.shape) - res).backward()

for name,i in ec.named_parameters():
    print(name)
    print(i.requires_grad)
    print(i.grad)

mlp.0.weight
True
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
mlp.0.bias
True
tensor([0.])
mlp.2.weight
True
tensor([[0.]])
mlp.2.bias
True
tensor([-10.])


torch.Size([10, 1])

In [3]:
def check(model):
    for i in range(len(model.model.model)):
        print('i', i)
        nodes = model.model.model[i].edge_conv.edge_index
        print(nodes.shape)
        print("k_degree", model.model.model[i].edge_conv.k_degree)
        print(torch.unique(nodes[0]).shape)
        print(torch.unique(nodes[1]).shape)


check(model_classic)
print('------')
check(model_dcg)

i 0
torch.Size([2, 455])
k_degree 0
torch.Size([455])
torch.Size([455])
i 1
torch.Size([2, 455])
k_degree 1
torch.Size([323])
torch.Size([455])
i 2
torch.Size([2, 910])
k_degree 2
torch.Size([423])
torch.Size([455])
i 3
torch.Size([2, 1365])
k_degree 3
torch.Size([441])
torch.Size([455])
i 4
torch.Size([2, 1820])
k_degree 4
torch.Size([447])
torch.Size([455])
------
i 0
torch.Size([2, 455])
k_degree 1
torch.Size([419])
torch.Size([455])
i 1
torch.Size([2, 455])
k_degree 1
torch.Size([313])
torch.Size([455])
i 2
torch.Size([2, 910])
k_degree 2
torch.Size([399])
torch.Size([455])
i 3
torch.Size([2, 1365])
k_degree 3
torch.Size([424])
torch.Size([455])
i 4
torch.Size([2, 1820])
k_degree 4
torch.Size([446])
torch.Size([455])


In [5]:
res_dcg

{'loss': tensor(0.6780, grad_fn=<NllLossBackward0>),
 'accuracy': tensor(0.6286),
 'f1': 0.7719298245614035,
 'avr_precision': tensor(0.3953),
 'aucroc': tensor(0.2922),
 'temperatue_0': tensor([1.]),
 'temperatue_1': tensor([1.]),
 'temperatue_2': tensor([1.]),
 'temperatue_3': tensor([1.]),
 'temperatue_4': tensor([1.])}

tensor([-20079384.])
tensor([-0.1329])
None
None


/opt/anaconda3/envs/lev_eg/lib/python3.9/site-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068185/work/build/aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


In [13]:
torch.Tensor([0.2233123123123121])[0]

tensor(0.2233)

In [4]:
idx = 3
a = model_dcg.model.model[idx].edge_conv.edge_index[:, torch.argsort(model_dcg.model.model[idx].edge_conv.edge_index[1])]
a[:,:10]

tensor([[330, 282,   0, 223, 128,   1,   2,  45,  78,   3],
        [  0,   0,   0,   1,   1,   1,   2,   2,   2,   3]])

In [5]:
model_dcg.model.model[idx].edge_conv.probs

tensor([[1.0000e+00, 1.0000e-06, 1.0000e-06,  ..., 1.0000e-06, 1.0000e-06,
         1.0000e-06],
        [1.0000e-06, 1.0000e+00, 1.0000e-06,  ..., 1.0000e-06, 1.0000e-06,
         1.0000e-06],
        [1.0000e-06, 1.0000e-06, 1.0000e+00,  ..., 1.0000e-06, 1.0000e-06,
         1.0000e-06],
        ...,
        [1.0000e-06, 1.0000e-06, 1.0000e-06,  ..., 1.0000e+00, 1.0000e-06,
         1.0000e-06],
        [1.0000e-06, 1.0000e-06, 1.0000e-06,  ..., 1.0000e-06, 1.0000e+00,
         1.0000e-06],
        [1.0000e-06, 1.0000e-06, 1.0000e-06,  ..., 1.0000e-06, 1.0000e-06,
         1.0000e+00]], grad_fn=<AddBackward0>)

AttributeError: 'DynamicEdgeConv_DCG' object has no attribute 'e'

In [4]:
model_dcg.model.model[0].edge_conv.temperature

Parameter containing:
tensor([1.], requires_grad=True)

In [6]:
nodes_emb = model_dcg.model.model[idx].edge_conv.nodes_emb

In [7]:
torch.exp(-1 * torch.sum(nodes_emb - nodes_emb.unsqueeze(1), dim=-1)**2)

tensor([[1.0000e+00, 0.0000e+00, 2.7640e-13,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 1.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.7640e-13, 0.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 1.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00]])

In [28]:
modules

[Sequential(
   (0): Linear(in_features=1, out_features=1, bias=True)
   (conv_1): Linear(in_features=1, out_features=1, bias=True)
 ),
 Sequential(
   (0): Linear(in_features=1, out_features=1, bias=True)
   (conv_1): Linear(in_features=1, out_features=1, bias=True)
 )]